<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Asset Management

### Combining ML with MVP

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [training@tpq.io](mailto:trainin@tpq.io) | [@dyjh](http://twitter.com/dyjh)

## Expected Return Predictions

Topics of interest include:

* deep learning-based predictions
* mean-variance optimal portfolios

# Imports & Configurations

In [ ]:
!git clone https://github.com/tpq-classes/python_for_asset_management.git
import sys
sys.path.append('python_for_asset_management')


In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
from pylab import mpl, plt

In [ ]:
np.random.seed(1000)
plt.style.use('seaborn-v0_8')
pd.set_option('display.precision', 4)
pd.set_option('display.float_format', '{:.4f}'.format)
np.set_printoptions(suppress=True, precision=4)
%config InlineBackend.figure_format = 'svg'

## The Data

In [ ]:
raw = pd.read_csv('http://hilpisch.com/aiif_eikon_eod_data.csv',
                      index_col=0, parse_dates=True)

In [ ]:
noa = 3

In [ ]:
symbols = raw.columns[:noa]
symbols

In [ ]:
data = pd.DataFrame(raw[symbols])

In [ ]:
rets = np.log(data / data.shift(1))

In [ ]:
rets.dropna(inplace=True)

In [ ]:
rets.head()

In [ ]:
lags = 3

In [ ]:
def create_lags(rets):
    global cols
    cols = []
    for sym in rets.columns:
        for lag in range(1, lags + 1):
            col = f'{sym}_lag_{lag}'
            rets[col] = rets[sym].shift(lag)
            cols.append(col)

In [ ]:
create_lags(rets)

In [ ]:
cols

In [ ]:
rets.dropna(inplace=True)

## Predicting Returns

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, accuracy_score

In [ ]:
model = MLPRegressor(hidden_layer_sizes=2 * [32,],
                    random_state=100, shuffle=False)

In [ ]:
weeks = pd.date_range(rets.index[0], rets.index[-1], freq='w')

In [ ]:
start = 51

In [ ]:
%%time
rets_ = pd.DataFrame()
for w in weeks[start:-1]:
    train = rets.loc[:w].copy()
    mu, std = train.mean(), train.std()
    train_ = (train - mu) / std
    model.fit(train_[cols], train[symbols])
    test = rets.loc[w:w + weeks.freq].copy()
    test_ = (test - mu) / std
    rets_ = pd.concat([rets_, pd.DataFrame(model.predict(test_[cols]),
                                  columns=symbols, index=test.index)])  

## MVP Optimal Portfolios

In [ ]:
from scipy.optimize import minimize

In [ ]:
from risk_measures import *

In [ ]:
bnds = len(symbols) * [(-1, 1)]

In [ ]:
cons = {'type': 'eq', 'fun': lambda weights: weights.sum() - 1}

In [ ]:
w = rets_.copy()

In [ ]:
def sharpe(weights, r, re):
    port_ret = np.dot(r, weights)
    port_vol = portfolio_volatility(weights, re)
    return port_ret / port_vol

In [ ]:
%%time
for pred in rets_.iloc[:].iterrows():
    r = pred[1].values * 252
    re = rets.loc[:pred[0]][symbols]
    opt = minimize(lambda w: -sharpe(w, r=r, re=re),
                  np.ones(len(symbols)), bounds=bnds,
                         constraints=cons)
    w.loc[pred[0]] = opt['x']

In [ ]:
opt['x']

In [ ]:
w.iloc[:15].plot();

In [ ]:
rets_.iloc[0]

In [ ]:
w.iloc[0]

In [ ]:
rets.loc[w.index[0]][symbols]

In [ ]:
w.iloc[0] * rets.loc[w.index[0]][symbols]

In [ ]:
# w.index

In [ ]:
strat = w * rets.loc[w.index[0]:][symbols]

In [ ]:
strat.dropna(inplace=True)

In [ ]:
strat.head()

In [ ]:
strat.cumsum().apply(np.exp).plot(label='Portfolio')
plt.legend();  # out-of-sample performance

In [ ]:
strat.cumsum().sum(axis=1).apply(np.exp).plot(label='Portfolio')
plt.legend();  # out-of-sample performance

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>